In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List, cast, Literal, Tuple, Dict
from sklearn.multioutput import RegressorChain
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from xgboost import XGBRegressor
import lightgbm as lgb
import optuna

**LOAD DATA WITH CORRECT TYPES**

In [2]:
stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
holidays_events = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv",dtype={
            "type": "category",
            "locale": "category",
            "locale_name": "category",
            "description": "category",
            "transferred": "bool",
            "date": "period[D]"
        }
)
store_sales = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv",
        dtype={
            "store_nbr": "category",
            "family": "category",
            "sales": "float32",
            "onpromotion": "uint32",
            "date": "period[D]"
        }
)
query = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv",
        dtype={
            "store_nbr": "category",
            "family": "category",
            "onpromotion": "uint32",
            "date": "period[D]"
        }
)
oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv", 
        dtype={            
            "date": "period[D]"
        }
)
transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

**ADDITIONAL HELPER FUNCTIONS**

In [3]:
main_index: List[str] = ["date", "store_nbr", "family"]
secondary_index: List[str] = ["store_nbr", "family"]
unique_families = store_sales["family"].unique()
unique_stores = store_sales["store_nbr"].unique()

def timeline(reset = False):
    global timeline_index
    if reset is True:
        timeline_index = 1
    else :
        try:
            timeline_index += 1
        except Exception:
            timeline_index = 1
        
    print("{}.Current time: {}".format(timeline_index,datetime.now().strftime("%H:%M:%S")))
timeline()

1.Current time: 07:58:32


In [4]:
def make_mw_in_groups(
    df: pd.DataFrame,
    groupby: List[str] = [],
    column: str = "",
    window: List[int] | int = 30,
    center: List[bool] | bool  = False,
    min_periods: List[int] | int  = 1,
    aggregator: List[Literal["mean","sum","median","std","var"]] | Literal["mean","sum","median","std","var"] = "mean",
    name: str | None = None,
) -> pd.DataFrame:
    df = df.copy(deep=True)
    if name is None:
        name = column
    
    if isinstance(window,int):
        window = [window]
        
    window = list(filter(lambda x : x != 0, window))
    if len(window) == 0 :
        raise ValueError("Window value must be non-zero!")
    if isinstance(center,bool):
        center = [center] * len(window)
    if isinstance(min_periods,int):
        min_periods = [min_periods]*len(window)
    if isinstance(aggregator,str):
        aggregator = [aggregator]*len(window)
        
    
    def create_mw_columns(group):
        ma_group = pd.DataFrame(index=group.index)
        for index, val in enumerate(window):
            type_name = "lag" if val > 0 else "lead"
            if val < 0 :
                ma_group[f"{name}_{type_name}_{aggregator[index]}_{-val}"] = group[column].rolling(window=-val, center=center[index], min_periods=min_periods[index]).aggregate(aggregator[index]).shift(val)
            else:
                ma_group[f"{name}_{type_name}_{aggregator[index]}_{val}"] = group[column].shift(1).rolling(window=val, center=center[index], min_periods=min_periods[index]).aggregate(aggregator[index])
        
        return ma_group
    
    return cast(pd.DataFrame,df.reset_index(groupby).groupby(groupby, observed=True).apply(create_mw_columns, include_groups=False).reset_index(groupby).set_index(groupby, append=True).sort_index())


def make_shift_in_groups(
    df: pd.DataFrame,
    groupby: List[str] = [],
    column: str = "",
    shift: List[int] | int = 1,
    name : str | None = None,
) -> pd.DataFrame:
    df = df.copy(deep=True)
    if name is None:
        name = column
        
    if isinstance(shift,int):
        shift = [shift]
    
    shift = list(filter(lambda el: el != 0, shift))
    
    if len(shift) == 0:
        raise ValueError(
            "Shift value must be non-zero!"
        )

    def create_shifted_columns(group):
        shifted_group = pd.DataFrame(index=group.index)
        for val in shift:

            shifted_group[f"{name}_{'lead' if val < 0 else 'lag'}_{abs(val)}"] = (
                group[column].shift(val)
            )

        return shifted_group

    shifted_df = cast(
        pd.DataFrame,
        df.reset_index(groupby)
        .groupby(groupby, observed=True)
        .apply(create_shifted_columns, include_groups=False).reset_index(groupby).set_index(groupby, append=True).sort_index(),
    )

    return shifted_df

In [5]:
TRAIN_START = "2014-01-01"
TRAIN_END = "2017-07-29"

TEST_DAY = "2017-07-30"

X_START = "2014-01-01"
X_END = "2017-07-30"

QUERY_DAY = "2017-08-15"

N_STEP_PREDICTION = 16

**DATA PREPARATION**

In [6]:
def combine_data(store_sales, query, oil, holidays_events) -> pd.DataFrame:
    store_sales = store_sales.copy(deep=True)
    query = query.copy(deep=True)
    oil = oil.copy(deep=True)
    holidays_events.copy(deep=True)
    
    data = pd.concat([
        store_sales.set_index(main_index),
        query.set_index(main_index)
    ], axis=0 )
    
    holidays_to_consider = holidays_events[
        (holidays_events["transferred"].eq(False))
        & holidays_events["locale"].isin(["National"])
    ].drop_duplicates(keep="first", subset=["date"])
    
    all_periods = pd.period_range("2013-01-01", "2017-08-31")
    oil_prices = (
        oil.set_index("date")
        .reindex(all_periods)
        .rename_axis("date").ffill().bfill()
        .rename(columns={"dcoilwtico":"oil"})
    )
    
    data_combined = (
        data.reset_index(secondary_index)
        .join(oil_prices)
    )
    data_combined["is_holiday"] = data_combined.index.get_level_values("date").isin(holidays_to_consider["date"])
    data_combined = data_combined.set_index(secondary_index, append=True)
    return data_combined

**FEATURE ENGINEERING**

In [7]:
def prepare_days_since_last_paycheck(date: pd.PeriodIndex) -> List[int]:

    days_since_paycheck = [1] * len(date)
    for i, period in enumerate(date):
        if period.day == 15 or period.day == period.days_in_month:
            continue
        elif period.day < 15:
            days_since_paycheck[i] = period.day + 1
        else:
            days_since_paycheck[i] = period.day - 15 + 1

    return days_since_paycheck


def engineer_features(
    data : pd.DataFrame,
    sales_shift : List[int] = [1,2,4],
    sales_ma_window : List[int] = [7,14,28],
    dow_ma_weeks_window : List[int] = [10],
    oil_ma_window : List[int] = [-7, -16]
) -> pd.DataFrame:
    data = data.copy(deep=True)
    data["year"] = data.index.get_level_values("date").year
    data["month"] = data.index.get_level_values("date").month
    data["day_of_week"] = data.index.get_level_values("date").day_of_week
    data["end_of_year"] = data.index.get_level_values("date").month >= 11
    data["quarter"] = data.index.get_level_values("date").quarter
    data["family_cat"] = data.index.get_level_values("family")
    data["store_cat"] = data.index.get_level_values("store_nbr")
    
#     data["days_since_last_paycheck"] = prepare_days_since_last_paycheck(date=data.index.get_level_values("date"))
#     data["eartquake_impact"] = data.index.get_level_values("date").isin(pd.period_range("2016-04-16", periods=90))
    
    grouped_lags_leads: List[pd.DataFrame] = [
        make_shift_in_groups(
            df = data,
            groupby = secondary_index,
            column = "sales",
            shift = sales_shift
        ),
        make_mw_in_groups(
            df = data,
            groupby = secondary_index,
            column = "sales",
            window = sales_ma_window,
            aggregator = "mean",
            center = False
        ),
        make_mw_in_groups(
            df = data.set_index("day_of_week",append=True),
            groupby = ["store_nbr","family","day_of_week"],
            column = "sales",
            # --- NOTICE ---
            # window references the weeks
            window = dow_ma_weeks_window,
            aggregator = "mean"
        ).reset_index("day_of_week", drop=True),
        make_mw_in_groups(
            df = data,
            groupby = secondary_index,
            column = "oil",
            window = oil_ma_window
        ),
    ]
    
    data_lag_lead = data.join(grouped_lags_leads)
    
    category_cols = ["year","month","day_of_week","end_of_year","quarter","family_cat","store_cat"]
    data_lag_lead[category_cols] = data_lag_lead[category_cols].astype("category")
    
    return data_lag_lead
        


**PREPARE TRAIN & TEST SETS**

In [8]:
def split_train_test(
    df : pd.DataFrame,
    train_start: str,
    train_end: str,
    test_day: str,
    verboose: bool = True
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    
    df = df.copy()
    if verboose is True:
        display(df.columns)
    
    col_x_drop = ["id","sales"]
    target_cols = ["sales"]
    
    X_train = df.loc[train_start: train_end].drop(columns=col_x_drop)
    X_test = df.loc[test_day: test_day].drop(columns=col_x_drop)
    
    y = make_shift_in_groups(
        df = df,
        groupby = secondary_index,
        column = "sales",
        shift = [-i for i in range(1, N_STEP_PREDICTION+1)]
    )
    y_train = y.loc[train_start: train_end]
    y_test = y.loc[test_day: test_day]
    
    X_train_d = pd.get_dummies(X_train, drop_first = True)
    X_test_d = pd.get_dummies(X_test, drop_first = True)
    return X_train_d, X_test_d, y_train, y_test
    

In [9]:
def create_model(**params) :
    # --- NOTICE ---
    # It will allow for easy parameters manipulation and model changes
    model = RegressorChain(XGBRegressor(**params))
#     model = lgb.LGBMRegressor(**params)
    return model
    
def validate_model(X_train, X_test, y_train, y_test) -> Tuple[pd.DataFrame, float]:
    model = create_model().fit(X_train, y_train)
    y_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y_train.columns).clip(0.0)
    
    rsmle = mean_squared_log_error(y_test, y_pred)
    
    return y_pred, rsmle


In [10]:
def rerun_model_test(verboose : bool = True, feature_adjustments : Dict[str,List[int]] = {}):
    data = combine_data(store_sales, query, oil, holidays_events)
    if verboose is True:
        display("COMBINED DATA", data)
    
    
    eng_feat_set = engineer_features(data, **feature_adjustments)
    if verboose is True:
        display("ENGINEERED FEATURES", eng_feat_set)
    
    X_train, X_test, y_train, y_test = split_train_test(eng_feat_set, TRAIN_START, TRAIN_END, TEST_DAY, verboose=verboose)
    if verboose is True:
        display("TRAIN/TEST SPLIT", X_train, X_test, y_train,y_test)
    
    
    if verboose is True:
        display("VALIDATING MODEL...")
    
    timeline()
    y_test_pred, rsmle = validate_model(X_train, X_test, y_train, y_test)
    print(f"TESTED FOR: {feature_adjustments}, RSMLE: {rsmle}")
    timeline()
    
    if verboose is True:
        display(y_test_pred)



In [11]:
feature_adjustments_list : List[Dict[str,List[int]]] = [
#     {},
]
    
for feature_adjustments in feature_adjustments_list:
    rerun_model_test(verboose=False, feature_adjustments=feature_adjustments )

***LOGBOOK***

**1.Initial setup without one-hot encodings:** RSMLE: 0.6812447830583148

**2.Initial setup with all one-hot encodings:** RSMLE: 0.6145283461216959

**3.Initial setup without DOWM_10:** RSMLE: 0.6559099951029017

**4.V2 but no family and store_nbr:** RSMLE: 0.6302813566155643

**5.Initial setup with 1-16-steps lead onpromotion:** RSMLE: 0.672006775918979 | ~8min

**6.V4 without end_of_year:** RSMLE: 0.6715956378282983 | ~6min

**7.V4 with eartquake_impact:** RSMLE: 0.6650935588175516 | ~7min

**8.V4 without days_since_last_paycheck:** RSMLE: 0.5112336225425305 | ~7min

**9.V8 with quarter:** RSMLE: 0.4681872772081537 | ~7min

**10.V9 with start_year:** RSMLE: 0.4963768445333584 | ~7min

**11.V10 training starts from 2016-01-01:** RSMLE: 0.7411192084551556 | ~3min

**12.V8 with sales_lag_3:** RSMLE: 0.46908549576706216 | ~6min

**13.V8 with sales_lag_4:** RSMLE: 0.4578231468785362 | ~7min

**14.V8 with best_hyper_parameters:** RSMLE: 0.3027026586082895 | ~1h

**15.V13 with family and store categories**


**OPTUNA OPTYMALIZATION**

In [12]:
def optymize_with_optuna():
    data = combine_data(store_sales, query, oil, holidays_events)
    eng_feat_set = engineer_features(data)    
    X_train, X_test, y_train, y_test = split_train_test(eng_feat_set, TRAIN_START, TRAIN_END, TEST_DAY)
    
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50),  # Number of trees in the ensemble
            "max_depth": trial.suggest_int("max_depth", 3, 10),  # Maximum depth of each tree
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),  # Learning rate
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),  # Subsample ratio of the training instances
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),  # Subsample ratio of columns when constructing each tree
            "gamma": trial.suggest_float("gamma", 0.01, 10.0, log=True),  # Minimum loss reduction required to make a further partition on a leaf node of the tree
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 100.0, log=True),  # L1 regularization term on weights
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 100.0, log=True),  # L2 regularization term on weights
            "min_child_weight": trial.suggest_float("min_child_weight", 1, 100, log=True),  # Minimum sum of instance weight (hessian) needed in a child
        }
        model = create_model(**params)
        model = model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred)

        return rmse

    study = optuna.create_study(direction="minimize", study_name="V9")
    study.optimize(objective, n_trials=8)

    best_params = study.best_params
    print((f"Best hyperparameters: {best_params}"))
    
# --- NOTICE ---
# Uncomment line below to run optymalization

# optymize_with_optuna()
best_xgb_hyper_parameters = {
    'n_estimators': 1000, 
    'max_depth': 9, 
    'learning_rate': 0.12605081311532235, 
    'subsample': 0.6245582839186816, 
    'colsample_bytree': 0.8404676393471824, 
    'gamma': 0.07728746653962874, 
    'reg_alpha': 3.2534719705809634, 
    'reg_lambda': 0.05386975318104558, 
    'min_child_weight': 5.771548202565809
} 


In [13]:
def prepare_prediction_for_output(y_pred: np.array, X_test : pd.DataFrame, y_train: pd.DataFrame, eng_feat_set: pd.DataFrame,  test_day : str, n_steps_prediction : int ) -> pd.DataFrame:
    y_pred_df = pd.DataFrame(y_pred, index=X_test.index, columns=y_train.columns).clip(0.0)
    y_pred_df.columns = pd.period_range(test_day, periods=n_steps_prediction+1)[1:]
    y_submission = y_pred_df.reset_index(secondary_index).melt(id_vars=secondary_index,var_name="date",value_name="sales").set_index(main_index).join(eng_feat_set["id"]).reindex(columns=["id","sales"])
    
    return y_submission

In [14]:
 def rerun_model_submission(name:str = "multi_submission.csv", hyper_params : Dict[str,int | float] = {}, feature_adjustments : Dict[str,List[int]] = {} ):
    data = combine_data(store_sales, query, oil, holidays_events) 
    
    eng_feat_set = engineer_features(data, **feature_adjustments) 
    
    X_train, X_test, y_train, y_test = split_train_test(eng_feat_set, X_START, X_END, QUERY_DAY, verboose=False)
    
    model = create_model(**hyper_params).fit(X_train, y_train)
    y_submission_df = prepare_prediction_for_output(model.predict(X_test), X_test=X_test, y_train=y_train, eng_feat_set=eng_feat_set, test_day= QUERY_DAY, n_steps_prediction = N_STEP_PREDICTION)
    
    y_submission_df.to_csv(name, index=False)
    return y_submission_df
    

In [15]:
# sales_shift
# sales_ma_window 
# dow_ma_weeks_window 
# oil_ma_window
submission_generation_list : List[Dict] = [
    {
        "name": "v15.csv",
        "hyper_params":{},
        "feature_adjustments":{},
    },
#     {
#         "name": "v12.csv",
#         "hyper_params":{},
#         "feature_adjustments":{
#             "sales_shift":[1,2,3]
#         },
#     },
#     {
#         "name": "v13_with_oil_ma_m16.csv",
#         "hyper_params":{},
#         "feature_adjustments":{
#             "oil_ma_window": [-16]
#         },
#     },
#     {
#         "name": "v13_with_sales_ma_90.csv",
#         "hyper_params":{},
#         "feature_adjustments":{
#             "sales_ma_window": [7,14,21]
#         },
#     },
    {
        "name": "v15_with_best_hyper_parameters.csv",
        "hyper_params":best_xgb_hyper_parameters,
        "feature_adjustments":{},
    },
]
    
for setting in submission_generation_list:
    rerun_model_submission(**setting)